# Feature Engineering
[Kaggle's Feature Engineering](https://www.kaggle.com/learn/feature-engineering)

## Baseline Model
[Kaggle's Baseline Model](https://www.kaggle.com/matleonard/baseline-model)

[Kaggle's Baseline Model: Exercise](https://www.kaggle.com/matleonard/baseline-model)

## Categorical Encodings
1. Label Encoding
2. One Hot Encoding
3. Count Encoding
4. Target Encoding
5. CatBoost Encoding
6. Singular Value Decomposition

[Kaggle's Categorical Encodings](https://www.kaggle.com/matleonard/categorical-encodings)

[Kaggle's Categorical Encodings: Exercise](https://www.kaggle.com/aubreyjohn/exercise-categorical-encodings/edit)

## Feature Generation
Creating new features from the raw data is one of the best ways to improve your model.

The features you create are different for every dataset, so it takes a bit of creativity and experimentation. 

You can have access to multiple tables with relevant data that you can use to create new features.

You can make new columns by:

- Combining categorical columns (the combined columns are called interactions).
- Making calculations based on numerical columns.
 
Use **.rolling()** to calculate number of items in a period e.g last 7 days. Takes in a series with timestamp as index and indices as values.

If you want a window that always starts at the first row but expands as you get further in the data, you can use the **.expanding** methods for this.


Calculate time since last item in the same category. A handy method for performing operations within groups is to use **.groupby**. then **.transform**. The **.transform**. method takes a function then passes a series or dataframe to that function for each group. This returns a dataframe with the same indices as the original dataframe. In our case, we'll perform a groupby on "category" and use transform to calculate the time differences for each category.

def time_since_last_project(series):
    # Return the time in hours
    return series.diff().dt.total_seconds() / 3600.
    
df = ks[['category', 'launched']].sort_values('launched')

timedeltas = df.groupby('category').transform(time_since_last_project)


Transforming numerical values. Some models work better when the features are normally distributed. Common choices for this are the **square root** and **natural logarithm**.

The **log transformation** won't help if your model is tree-based since tree-based models are scale invariant. However, this should help if you have a **linear model** or **neural network**.

Other transformations include **squares** and other **powers**, **exponentials**, etc. These might help the model discriminate, like the **kernel trick for SVMs**. 

Again, it takes a bit of experimentation to see what works.

[Kaggle's Feature Generation](https://www.kaggle.com/learn/feature-engineering)

[Kaggle's Feature Generation: Exercise](https://www.kaggle.com/aubreyjohn/exercise-feature-generation/edit)

 One method is to create a bunch of new features and later choose the best ones with **feature selection algorithms**.

## Feature Selection
[Kaggle's Feature Selection](https://www.kaggle.com/matleonard/feature-selection)

[Kaggle's Feature Selection: Exercise](https://www.kaggle.com/aubreyjohn/exercise-feature-selection/edit)

Often you'll have hundreds or thousands of features after various encodings and feature generation. This can lead to two problems. First, the more features you have, the more likely you are to overfit to the training and validation sets. This will cause your model to perform worse at generalizing to new data.

Secondly, the more features you have, the longer it will take to train your model and optimize hyperparameters. Also, when building user-facing products, you'll want to make inference as fast as possible. Using fewer features can speed up inference at the cost of predictive performance.

To help with these issues, you'll want to use feature selection techniques to keep the most informative features for your model.

1. Univariate Feature Selection

The simplest and fastest methods are based on univariate statistical tests. For each feature, measure how strongly the target depends on the feature using a statistical test like  **χ2**  or **ANOVA**.

From the scikit-learn feature selection module, **feature_selection.SelectKBest** returns the K best features given some scoring function.

For a classification problem, the module provides three different scoring functions:  **χ2** , **ANOVA F-value**, and the **mutual information score**. The F-value measures the linear dependency between the feature variable and the target. This means the score might underestimate the relation between a feature and the target if the relationship is nonlinear. The **mutual information score** is nonparametric and so can capture nonlinear relationships.

With **SelectKBest**, we define the number of features to keep, based on the score from the scoring function. Using **.fit_transform(features, target)** we get back an array with only the selected features.

2. L1 regularization

Univariate methods consider only one feature at a time when making a selection decision. Instead, we can make our selection using all of the features by including them in a linear model with **L1 regularization**. This type of regularization (sometimes called **Lasso**) penalizes the absolute magnitude of the coefficients, as compared to **L2 (Ridge) regression** which penalizes the square of the coefficients.

As the strength of regularization is increased, features which are less important for predicting the target are set to 0. This allows us to perform feature selection by adjusting the regularization parameter. We choose the parameter by finding the best performance on a hold-out set, or decide ahead of time how many features to keep.

For regression problems you can use **sklearn.linear_model.Lasso**, or **sklearn.linear_model.LogisticRegression** for classification. These can be used along with **sklearn.feature_selection.SelectFromModel** to select the non-zero coefficients. Otherwise, the code is similar to the univariate tests.


In general, feature selection with L1 regularization is more powerful the univariate tests, but it can also be very slow when you have a lot of data and a lot of features. Univariate tests will be much faster on large datasets, but also will likely perform worse.